# This notebook contains code for adding amide bond caps to alpha-amino acids.  This concept was described in Ilardo et al. (2015) (https://doi.org/10.1038/srep09414) using MOLGEN; current approach using RDKit used in Mayer-Bacon and Freeland (2021) (https://doi.org/10.1016/j.jtbi.2021.110661).

## Enter input and output filenames (must be in .sdf format) in cell below

In [ ]:
in_file = "C:\\Users\\umbci\\Desktop\\MAYGEN\\Libraries\\VAIL_S.sdf"
out_file = "C:\\Users\\umbci\\Desktop\\MAYGEN\\Libraries\\VAIL_S_cap.sdf"

### If you encounter a SyntaxError mentioning a unicode error, make sure there are no single backslashes (\\) in the file names.  Change any single backslashes to double backslashes (\\\\) or forward slashes (/).

## Module imports below

In [ ]:
from rdkit import Chem
from rdkit.Chem import rdChemReactions

## Function definitions below

In [ ]:
def aaCapping(mol):
    """Caps the N- and C-termini of an alpha-amino acid (add an acetyl group and N-methyl group, respectively).
    
    Intended to mimic an amino acid in the polypeptide context, i.e. with amide bonds instead of free amine and carboxyl groups.
    
    Parameters
    ----------
    mol : RDKit Mol object
    
    Returns
    -------
    RDKit Mol object with capped C- and N-termini if input is an alpha-amino acid; returns input molecule if not an alpha-amino acid

    """
    # set patterns corresponding to amino acid skeletons
    patt_1a = Chem.MolFromSmarts('[NX3:1][CX4H]([*:3])[CX3:2](=[OX1])[O]') # monosubstituted alpha-AA
    patt_gly = Chem.MolFromSmarts('[NX3:1][CX4H2][CX3:2](=[OX1])[O]') # glycine
    patt_sec_gly = Chem.MolFromSmarts('[*:3][NX3:1][CX4H2][CX3:2](=[OX1])[O]') # N-substituted glycines
    patt_sec_amine = Chem.MolFromSmarts('[*:3][NX3:1][CX4H]([*:4])[CX3:2](=[OX1])[O]') # N-substituted, monosubstituted
    # recalculate implicit valence and ring info before transformation
    mol.UpdatePropertyCache()
    if mol.HasSubstructMatch(patt_1a):
        cap = rdChemReactions.ReactionFromSmarts('[NX3:1][CX4H]([*:3])[CX3:2](=[OX1])[O]>>[NX3:1](C(=O)C)[CX4H]([*:3])[CX3:2](=[OX1])NC')
    elif mol.HasSubstructMatch(patt_gly):
        cap = rdChemReactions.ReactionFromSmarts('[NX3:1][CX4H2][CX3:2](=[OX1])[O]>>[NX3:1](C(=O)C)[CX4H2][CX3:2](=[OX1])NC')
    elif mol.HasSubstructMatch(patt_sec_gly):
        cap = rdChemReactions.ReactionFromSmarts('[*:3][NX3:1][CX4H2][CX3:2](=[OX1])[O]>>[*:3][NX3:1](C(=O)C)[CX4H2][CX3:2](=[OX1])NC')
    elif mol.HasSubstructMatch(patt_sec_amine):
        cap = rdChemReactions.ReactionFromSmarts('[*:3][NX3:1][CX4H]([*:4])[CX3:2](=[OX1])[O]>>[*:3][NX3:1](C(=O)C)[CX4H]([*:4])[CX3:2](=[OX1])NC')
    else:
        cap = None
    if cap != None:
        return cap.RunReactant(mol,0)[0][0]
    else:
        return mol

## Main program

In [ ]:
# load in data file
data = Chem.SDMolSupplier(in_file)
# create file writer
w = Chem.SDWriter(out_file)
# loop through each input structure, cap and write to file
for mol in data:
    mol_cap = aaCapping(mol)
    w.write(mol_cap)
w.close()